# Price Visualisation

Interactive candlestick charts with dynamic zoom-based resolution switching.

## Data generation

```bash
# Single week (quick test)
./build/qrsdp_run --seed 42 --days 5 --seconds 23400

# Full year (252 trading days, ~580M events, ~6.8 GB)
./build/qrsdp_run --seed 42 --days 252 --seconds 23400
```

In [1]:
import sys
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
from IPython.display import display

import qrsdp_reader as reader
import book_replay as replay
import ohlc

In [2]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")
SINGLE_DAY_FILE = None  # auto-detect first day from manifest

manifest = reader.load_manifest(RUN_DIR)
symbols = reader.manifest_symbols(manifest)
multi_security = len(symbols) > 0

if multi_security:
    sessions = manifest["securities"][0]["sessions"]
    print(f"Run directory: {RUN_DIR}  (multi-security: {', '.join(symbols)})")
else:
    sessions = manifest["sessions"]
    print(f"Run directory: {RUN_DIR}")

print(f"Sessions: {len(sessions)}, dates: {sessions[0]['date']} .. {sessions[-1]['date']}")
print(f"Seed: {manifest['base_seed']}, strategy: {manifest['seed_strategy']}")

Run directory: ../output/run_42
Sessions: 252, dates: 2026-01-02 .. 2026-12-21
Seed: 42, strategy: sequential


## Single-Day Deep Dive

Load one day, replay the order book, and show a zoomable candlestick chart
that dynamically switches resolution as you zoom in/out.

In [3]:
day_file = SINGLE_DAY_FILE or (RUN_DIR / sessions[0]["file"])
header = reader.read_header(day_file)
events = reader.read_day(day_file)
print(f"Loaded {len(events):,} events from {day_file.name}")
print(f"p0 = {header['p0_ticks']}, tick_size = {header['tick_size']}, session = {header['session_seconds']}s")

Loaded 1,689,650 events from 2026-01-02.qrsdp
p0 = 10000, tick_size = 100, session = 23400s


In [4]:
book = replay.replay_book(
    events,
    p0_ticks=header["p0_ticks"],
    levels_per_side=header["levels_per_side"],
    initial_spread_ticks=header["initial_spread_ticks"],
    initial_depth=header["initial_depth"],
)
print(f"Mid range: {book['mid_ticks'].min():.1f} – {book['mid_ticks'].max():.1f}")
print(f"Spread range: {book['spread_ticks'].min()} – {book['spread_ticks'].max()}")

Mid range: 9960.5 – 10349.0
Spread range: 1 – 7


In [5]:
bars = ohlc.multi_resolution_ohlc(book["ts_ns"], book["mid_ticks"])
for label, df in bars.items():
    print(f"  {label:>4s}: {len(df):>6,} bars")

    1s: 23,400 bars
   10s:  2,340 bars
  1min:    390 bars
  5min:     78 bars


In [6]:
# --- Zoomable candlestick chart ---
initial_key, initial_bars = ohlc.select_resolution(bars, header["session_seconds"])

fig = go.FigureWidget(
    data=[
        go.Candlestick(
            x=initial_bars["time_s"],
            open=initial_bars["open"],
            high=initial_bars["high"],
            low=initial_bars["low"],
            close=initial_bars["close"],
            name="Mid-price",
        )
    ],
    layout=go.Layout(
        title=f"Mid-Price — {sessions[0]['date']} (resolution: {initial_key})",
        xaxis=dict(
            title="Time (seconds from session start)",
            rangeslider=dict(visible=True),
        ),
        yaxis=dict(title="Price (ticks)"),
        height=550,
        template="plotly_white",
    ),
)


def _on_zoom(layout, xrange):
    """Switch candlestick resolution when the user zooms."""
    if xrange is None or len(xrange) < 2:
        return
    visible_seconds = abs(xrange[1] - xrange[0])
    key, best = ohlc.select_resolution(bars, visible_seconds)
    with fig.batch_update():
        fig.data[0].x = best["time_s"]
        fig.data[0].open = best["open"]
        fig.data[0].high = best["high"]
        fig.data[0].low = best["low"]
        fig.data[0].close = best["close"]
        fig.layout.title.text = (
            f"Mid-Price — {sessions[0]['date']} (resolution: {key})"
        )


fig.layout.on_change(_on_zoom, "xaxis.range")
display(fig)

FigureWidget({
    'data': [{'close': {'bdata': ('AAAAAECVw0AAAAAAwJnDQAAAAAAAm8' ... 'AAAECnw0AAAAAAgLzDQAAAAAAAyMNA'),
                        'dtype': 'f8'},
              'high': {'bdata': ('AAAAAECew0AAAAAAAKfDQAAAAABAo8' ... 'AAAMCrw0AAAAAAQL/DQAAAAADA0cNA'),
                       'dtype': 'f8'},
              'low': {'bdata': ('AAAAAECCw0AAAAAAQIvDQAAAAABAi8' ... 'AAAMCIw0AAAAAAQKLDQAAAAABAucNA'),
                      'dtype': 'f8'},
              'name': 'Mid-price',
              'open': {'bdata': ('AAAAAACIw0AAAAAAQJXDQAAAAADAmc' ... 'AAAECLw0AAAAAAQKfDQAAAAABAvMNA'),
                       'dtype': 'f8'},
              'type': 'candlestick',
              'uid': '6f8e8f9f-2587-4f61-853b-1d511b191593',
              'x': {'bdata': ('QGeibdTBbD/RNupgDsByQGkbdTAHwI' ... 'iDOQD51UDbqIM5AETWQNuogzkAj9ZA'),
                    'dtype': 'f8'}}],
    'layout': {'height': 550,
               'template': '...',
               'title': {'text': 'Mid-Price — 2026-01-02 (resolution: 5m

### Bid-Ask Spread

In [7]:
# Down-sample spread for plotting (every 1000th event)
stride = max(1, len(book["ts_ns"]) // 20000)
t_s = book["ts_ns"][::stride] / 1e9
spread = book["spread_ticks"][::stride]

fig_spread = go.Figure(
    data=go.Scatter(x=t_s, y=spread, mode="lines", name="Spread"),
    layout=go.Layout(
        title=f"Bid-Ask Spread — {sessions[0]['date']}",
        xaxis=dict(title="Time (s)", rangeslider=dict(visible=True)),
        yaxis=dict(title="Spread (ticks)"),
        height=350,
        template="plotly_white",
    ),
)
fig_spread.show()

## Multi-Day Overview

Iterate all days, compute 5-min OHLC per day, and concatenate for a full-run
candlestick chart.

In [8]:
import pandas as pd

all_bars = []
for date, day_events in reader.iter_days(RUN_DIR):
    hdr = reader.read_header(RUN_DIR / [s["file"] for s in sessions if s["date"] == date][0])
    bk = replay.replay_book(
        day_events,
        p0_ticks=hdr["p0_ticks"],
        levels_per_side=hdr["levels_per_side"],
        initial_spread_ticks=hdr["initial_spread_ticks"],
        initial_depth=hdr["initial_depth"],
    )
    day_bars = ohlc.compute_ohlc(bk["ts_ns"], bk["mid_ticks"], ohlc.RESOLUTIONS["5min"])
    day_bars["date"] = date
    all_bars.append(day_bars)
    print(f"  {date}: {len(day_events):>10,} events -> {len(day_bars)} bars, "
          f"close={day_bars['close'].iloc[-1]:.1f}")

combined = pd.concat(all_bars, ignore_index=True)
combined["bar_idx"] = range(len(combined))
print(f"\nTotal: {len(combined):,} bars across {len(all_bars)} days")

  2026-01-02:  1,689,650 events -> 78 bars, close=10128.0
  2026-01-05:  1,686,335 events -> 78 bars, close=9780.5
  2026-01-06:  1,688,569 events -> 78 bars, close=9299.5
  2026-01-07:  1,687,213 events -> 78 bars, close=8556.5
  2026-01-08:  1,688,827 events -> 78 bars, close=8536.0
  2026-01-09:  1,686,873 events -> 78 bars, close=8612.5
  2026-01-12:  1,688,922 events -> 78 bars, close=7883.5
  2026-01-13:  1,688,382 events -> 78 bars, close=7274.5
  2026-01-14:  1,684,628 events -> 78 bars, close=7476.5
  2026-01-15:  1,687,903 events -> 78 bars, close=7754.0
  2026-01-16:  1,687,343 events -> 78 bars, close=7419.0
  2026-01-19:  1,684,937 events -> 78 bars, close=7883.5
  2026-01-20:  1,687,952 events -> 78 bars, close=7675.5
  2026-01-21:  1,688,439 events -> 78 bars, close=8069.5
  2026-01-22:  1,688,152 events -> 78 bars, close=7864.0
  2026-01-23:  1,687,788 events -> 78 bars, close=7791.5
  2026-01-26:  1,689,577 events -> 78 bars, close=8286.0
  2026-01-27:  1,685,813 event

In [9]:
fig_multi = go.Figure(
    data=go.Candlestick(
        x=combined["bar_idx"],
        open=combined["open"],
        high=combined["high"],
        low=combined["low"],
        close=combined["close"],
        name="Mid-price (5min bars)",
    ),
    layout=go.Layout(
        title="Multi-Day Mid-Price (5-min OHLC)",
        xaxis=dict(
            title="Bar index (across all days)",
            rangeslider=dict(visible=True),
        ),
        yaxis=dict(title="Price (ticks)"),
        height=550,
        template="plotly_white",
    ),
)
fig_multi.show()

## Multi-Security Overlay

If the manifest contains multiple securities (`--securities` flag), plot
each symbol's daily closing price on a single chart for comparison.

In [10]:
if multi_security:
    fig_overlay = go.Figure()
    colours = ["#1976D2", "#D32F2F", "#388E3C", "#F57C00", "#7B1FA2", "#0097A7"]

    for idx, symbol in enumerate(symbols):
        closes, dates = [], []
        for date, day_events in reader.iter_days(RUN_DIR, symbol=symbol):
            sec_info = next(s for s in manifest["securities"] if s["symbol"] == symbol)
            file_path = next(s["file"] for s in sec_info["sessions"] if s["date"] == date)
            hdr = reader.read_header(RUN_DIR / file_path)
            bk = replay.replay_book(
                day_events,
                p0_ticks=hdr["p0_ticks"],
                levels_per_side=hdr["levels_per_side"],
                initial_spread_ticks=hdr["initial_spread_ticks"],
                initial_depth=hdr["initial_depth"],
            )
            closes.append(float(bk["mid_ticks"][-1]))
            dates.append(date)

        fig_overlay.add_trace(go.Scatter(
            x=dates, y=closes,
            mode="lines+markers", name=symbol,
            line=dict(color=colours[idx % len(colours)], width=2),
            marker=dict(size=6),
        ))

    fig_overlay.update_layout(
        title="Multi-Security Daily Closing Price Overlay",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Price (ticks)"),
        height=500,
        template="plotly_white",
    )
    fig_overlay.show()
else:
    print("Single-security run — overlay chart skipped.")

Single-security run — overlay chart skipped.
